# YOLO v3 Object Detection

Let's see how to use the state of the art in object detection! Please make sure to watch the video, there is no code along here, since we can't reasonably train the YOLOv3 network ourself, instead we will use a pre-established version.

CODE SOURCE: https://github.com/xiaochus/YOLOv3

REFERENCE (for original YOLOv3): 

        @article{YOLOv3,  
              title={YOLOv3: An Incremental Improvement},  
              author={J Redmon, A Farhadi },
              year={2018} 
--------
----------
## YOU MUST WATCH THE VIDEO LECTURE TO PROPERLY SET UP THE MODEL AND WEIGHTS. THIS NOTEBOOK WON'T WORK UNLESS YOU FOLLOW THE EXACT SET UP SHOWN IN THE VIDEO LECTURE.
-------
-------

In [7]:
import os
import time
import cv2
import numpy as np
from model.yolo_model import YOLO

Using TensorFlow backend.


In [8]:
from collections import Counter

In [9]:
import pyttsx3
converter = pyttsx3.init()
converter.setProperty('rate', 150) 
converter.setProperty('volume', 0.7)

In [10]:
def process_image(img):
    """Resize, reduce and expand image.

    # Argument:
        img: original image.

    # Returns
        image: ndarray(64, 64, 3), processed image.
    """
    image = cv2.resize(img, (416, 416),
                       interpolation=cv2.INTER_CUBIC)
    image = np.array(image, dtype='float32')
    image /= 255.
    image = np.expand_dims(image, axis=0)

    return image

In [11]:
def get_classes(file):
    """Get classes name.

    # Argument:
        file: classes name for database.

    # Returns
        class_names: List, classes name.

    """
    with open(file) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]

    return class_names

In [12]:
def draw(image, boxes, scores, classes, all_classes):
    """Draw the boxes on the image.

    # Argument:
        image: original image.
        boxes: ndarray, boxes of objects.
        classes: ndarray, classes of objects.
        scores: ndarray, scores of objects.
        all_classes: all classes name.
    """
    height = np.size(image, 0)
    width = np.size(image, 1)
    sourceDirectory = r'E:\Computer-Vision-with-Python\06-Deep-Learning-Computer-Vision\06-YOLOv3'
    annotation = ET.Element('annotation')
    ET.SubElement(annotation, 'folder').text = 'kitty'
    ET.SubElement(annotation, 'filename').text = str(1)+'.jpg'
    ET.SubElement(annotation, 'segmented').text = '0'
    size = ET.SubElement(annotation, 'size')
    ET.SubElement(size, 'width').text = str(width)
    ET.SubElement(size, 'height').text = str(height)
    ET.SubElement(size, 'depth').text = '3'
    
    
    #names = []
    for box,cl, score in zip(boxes, classes, scores):
        if all_classes[cl] == 'person' or all_classes[cl] == 'car':
            x, y, w, h = box

            top = max(0, np.floor(x + 0.5).astype(int))
            left = max(0, np.floor(y + 0.5).astype(int))
            right = min(image.shape[1], np.floor(x + w + 0.5).astype(int))
            bottom = min(image.shape[0], np.floor(y + h + 0.5).astype(int))

            cv2.rectangle(image, (top, left), (right, bottom), (255, 0, 0), 2)
            cv2.putText(image, '{0} {1:.2f}'.format(all_classes[cl], score),
                        (top, left - 6),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.6, (0, 0, 255), 1,
                        cv2.LINE_AA)

            
            ob = ET.SubElement(annotation, 'object')
            ET.SubElement(ob, 'name').text = all_classes[cl]
            ET.SubElement(ob, 'pose').text = 'Unspecified'
            ET.SubElement(ob, 'truncated').text = '0'
            ET.SubElement(ob, 'difficult').text = '0'
            bbox = ET.SubElement(ob, 'bndbox')
            ET.SubElement(bbox, 'xmin').text = str(top)
            ET.SubElement(bbox, 'ymin').text = str(left)
            ET.SubElement(bbox, 'xmax').text = str(right)
            ET.SubElement(bbox, 'ymax').text = str(bottom)

    xml_str = ET.tostring(annotation)
    root = etree.fromstring(xml_str)
    xml_str = etree.tostring(root, pretty_print=True)
    xml_labels = str(1) + '.xml'
    save_path = os.path.join(sourceDirectory, xml_labels)
    with open(save_path, 'wb') as temp_xml:
        temp_xml.write(xml_str)
              
            
            
            #print('class: {0}, score: {1:.2f}'.format(all_classes[cl], score))
            #names.append(all_classes[cl])

       # print('box coordinate x,y,w,h: {0}'.format(box))
   # cnt = Counter()
   # for word in names:
   #   cnt[word] += 1
  #  print(dict(cnt))

In [13]:
def detect_image(image, yolo, all_classes):
    """Use yolo v3 to detect images.

    # Argument:
        image: original image.
        yolo: YOLO, yolo model.
        all_classes: all classes name.

    # Returns:
        image: processed image.
    """
    pimage = process_image(image)

   # start = time.time()
    boxes, classes, scores = yolo.predict(pimage, image.shape)
   # end = time.time()

   # print('time: {0:.2f}s'.format(end - start))

    if boxes is not None:
        draw(image, boxes, scores, classes, all_classes)

    return image

In [14]:
import time

In [15]:
def detect_video(yolo, all_classes):
    """Use yolo v3 to detect video.

    # Argument:
        video: video file.
        yolo: YOLO, yolo model.
        all_classes: all classes name.
    """
    video_path = os.path.join("videos", "test", video)
    camera = cv2.VideoCapture(video)
    cv2.namedWindow("detection", cv2.WINDOW_AUTOSIZE)

    # Prepare for saving the detected video
    sz = (int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    fourcc = cv2.VideoWriter_fourcc(*'mpeg')

    
    vout = cv2.VideoWriter()
    vout.open(os.path.join("videos", "res", video), fourcc, 20, sz, True)

    while True:
        res, frame = camera.read()

        if not res:
            break
    
            
        image = detect_image(frame, yolo, all_classes)
        
        cv2.imshow("detection", image)

        # Save the video frame by frame
        vout.write(image)

        if cv2.waitKey(0) & 0xff == ord('q'):
                break

    vout.release()
    camera.release()
    

In [16]:
yolo = YOLO(0.6, 0.5)
file = 'data/coco_classes.txt'
all_classes = get_classes(file)

c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [17]:
from lxml import etree
import xml.etree.cElementTree as ET

### Detecting Images

In [19]:
f = '1.jpg'
#path = 'images/test'+f
image = cv2.imread(f)
image = detect_image(image, yolo, all_classes)
cv2.imshow("test", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Detecting on Video

In [61]:
# # detect videos one at a time in videos/test folder    
video = 'car1.mp4'
detect_video(yolo, all_classes)